In [1]:
import os
from osgeo import gdal

# Download LCP files

In [23]:
#-*- coding:utf-8 -*-
import os, zipfile
import geopandas
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import Point
from pyproj import Proj, transform
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver import ActionChains
import time
from lxml import etree
import glob
from shapely.geometry import box
from fiona.crs import from_epsg
import pycrs
from pyproj import Geod
geod = Geod(ellps='GRS80')

#+proj=longlat +ellps=GRS80 +datum=NAD83 +no_defs 

cmd1 = '!pip install selenium'

driver_cnt = 0

wildfire = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
ffdriverpath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata'
initialdownloadfilepath = '/home/qweddww123/Downloads'
extractfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

fp = webdriver.FirefoxProfile('/home/qweddww123/.mozilla/firefox/0lnpvwfz.default-release')
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.dir', initialdownloadfilepath)
fp.set_preference('browser.helperApps.neverAsk.OpenFile', "application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
fp.set_preference("browser.helperApps.neverAsk.SaveToDisk","application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
driver = webdriver.Firefox(firefox_profile=fp, executable_path=ffdriverpath+'/geckodriver') #firefox driver 다운로드 받고, 다운로드 받은 파일 경로 지정

coords_list1 = []

lat_dis_per_1km = 0.009
lon_dis_per_1km = 0.01

for year in sorted(os.listdir(wildfire), key=int):
    print(year, '***************************************************************************************************************')
    if not os.path.isdir(extractfilepath+'/'+year):
        os.system('mkdir '+extractfilepath+'/'+year)
    for target_fire in sorted(os.listdir(wildfire+'/'+year), key=str):
        path = wildfire+'/'+year        
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi'):
            continue
        else:
            #print()
            #print('target_fire = ', target_fire)
            for wildfire_time in  sorted(os.listdir(path+'/'+target_fire), key=str):
                if(wildfire_time[0] != '.'):
                    #print('fire path = ', path+'/'+target_fire+'/'+wildfire_time)
                    for wildfire_shp in sorted(os.listdir(path+'/'+target_fire+'/'+wildfire_time), key=str):
                        extractedfile_targetpath = extractfilepath+'/'+year+'/'+target_fire
                        if not os.path.isdir(extractedfile_targetpath):
                            fileextension = os.path.splitext(wildfire_shp)[1]
                            if fileextension == '.shp':
                                if os.path.splitext(wildfire_shp)[0] == wildfire_time:
                                    #print(os.path.splitext(wildfire_shp)[0])
                                    #print('missing fire = ', extractedfile_targetpath)
                                    driver_cnt += 1
                                    filepath = path+'/'+target_fire+'/'+wildfire_time+'/'+wildfire_shp
                                    #read shpfile
                                    shpfile = geopandas.read_file(filepath)
                                    #print("filepath = ", filepath)

                                    #shpfile min, max coords
                                    try:
                                        x1 = float(shpfile.bounds.minx[0])
                                        y1 = float(shpfile.bounds.miny[0])
                                        x2 = float(shpfile.bounds.maxx[0])
                                        y2 = float(shpfile.bounds.maxy[0])
                                    except:
                                        break
                                    
                                    #x1 = float(shpfile.bounds.minx[0])
                                    #y1 = float(shpfile.bounds.miny[0])
                                    #x2 = float(shpfile.bounds.maxx[0])
                                    #y2 = float(shpfile.bounds.maxy[0])
                                    #print(y2,y1,x2,x1)

                                    #initial lon, lat: x1, y1
                                    az12, az21, dist = geod.inv(x1, y1, x2, y2)
                                    npts = 1
                                    del_s = dist/(npts+1)
                                    center = geod.fwd_intermediate(x1, y1, az12, npts=npts, del_s=del_s)
                                    #print("center point = ", center.lons[0], center.lats[0])

                                    az12, az21, dist = geod.inv(x1, y1, center.lons[0], center.lats[0])
                                    #print("azimuth from min point to the center = ", az12)

                                    new_x2, new_y2, max_backaz= geod.fwd(center.lons[0], center.lats[0], float(45.0), 21000*math.sqrt(2)/2.0)
                                    new_x1, new_y1, min_backaz= geod.fwd(center.lons[0], center.lats[0], float(-135.0), 21000*math.sqrt(2)/2.0)
                                    #print('max coords = ', new_x2, ", ", new_y2)
                                    #print('min coords = ', new_x1, ", ", new_y1)
                                    north = round(new_y2,8)
                                    south = round(new_y1,8)
                                    east = round(new_x2,8)
                                    west = round(new_x1,8)
                                    #print('north = ', north)
                                    #print('south = ', south)
                                    #print('east = ', east)
                                    #print('west = ', west)

                                    coords = str(north)+','+str(south)+','+str(east)+','+str(west)
                                    url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FIG61HZ,'
                                    #print("TEST url")
                                    #print(url)

                                    az12, az21, dist1 = geod.inv(center.lons[0], center.lats[0], new_x1, new_y1)
                                    #print("from center to min coords")
                                    #print(":", round(az12, 0), round(az21, 0))

                                    az12, az21, dist1 = geod.inv(new_x1, new_y1, new_x2, new_y2)
                                    #print("from new min to new max")
                                    #print(":", round(az12, 0), round(az21, 0))

                                    #print(round(dist1, 3))
                                    #dist = 21000*math.sqrt(2)
                                    #print(round(dist, 3))

                                    az12, az21, new_xdist = geod.inv(new_x1, new_y1, new_x2, new_y1)
                                    #print('new_xdist = ', round(new_xdist/1000, 0))
                                    az12, az21, new_ydist = geod.inv(new_x1, new_y1, new_x1, new_y2)
                                    #print('new_ydist = ', round(new_ydist/1000, 0))

                                    az12, az21, xdist = geod.inv(x1, y1, x2, y1)
                                    az12, az21, ydist = geod.inv(x1, y1, x1, y2)
                                    #print("wildfire xdist = ", xdist/1000)
                                    #print("wildfire ydist = ", ydist/1000)

                                    if xdist/1000 <= 10.0 and ydist/1000 <= 10.0:
                                        #print('shapefile size < 100.0km^2')
                                        coords = str(north)+','+str(south)+','+str(east)+','+str(west)
                                        #print('bounding coords = ', coords)

                                        #url의 위경도 수정 부분 코드 작성 필요
                                        #shp파일 읽어오고, 21km*21km에 해당하는 영역을 잡아내고 그걸 위,경도로 또 변환해서 아래 주소의 'AL=' 뒤에 입력
                                        #us_140 LCP
                                        #url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=LC137HZ,'
                                        #us_200 LCP
                                        #url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FE954HZ,'
                                        url = 'https://landfire.cr.usgs.gov/Website/distreq/RequestSummary.jsp?AL='+coords+'&PL=FIG61HZ,'
                                        #print(url)

                                        if driver_cnt == 1:
                                            driver.get(url)
                                            try:
                                                wait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//form')))
                                                driver.find_element_by_xpath("//form").click()
                                            except:
                                                time.sleep(10)
                                                driver_cnt = 0
                                                break

                                            coords_list1.append(coords)
                                            coords_list1.append(target_fire)
                                            coords_list1.append(wildfire_time)
                                        else:
                                            driver.current_window_handle
                                            #open a new tab
                                            #execute_url = 'window.open('+'"'+url+'"'+');'
                                            #driver.execute_script(execute_url)
                                            #driver.switch_to.window(driver.window_handles[driver_cnt-1])
                                            driver.get(url)
                                            #time.sleep(1)
                                            try:
                                                wait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//form')))
                                                driver.find_element_by_xpath("//form").click()
                                            except:
                                                time.sleep(10)
                                                #print('time.sleep(10)')
                                                driver_cnt = 0
                                                break

                                            coords_list1.append(coords)
                                            coords_list1.append(target_fire)
                                            coords_list1.append(wildfire_time)

                                        print('driver_cnt = ', driver_cnt)

                                        if driver_cnt == 1:
                                            #delay for downloading lcp files
                                            time.sleep(180)
                                            #close tabs
                                            for i in range(len(driver.window_handles)):
                                                driver.switch_to.window(driver.window_handles[0])
                                                driver.close()
                                            fp = webdriver.FirefoxProfile('/home/qweddww123/.mozilla/firefox/0lnpvwfz.default-release')
                                            fp.set_preference('browser.download.folderList', 2)
                                            fp.set_preference('browser.download.dir', initialdownloadfilepath)
                                            fp.set_preference('browser.helperApps.neverAsk.OpenFile', "application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
                                            fp.set_preference("browser.helperApps.neverAsk.SaveToDisk","application/octet-stream, multipart/x-zip, application/x-zip-compressed, application/zip, application/gzip, application/zip-compressed, text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml")
                                            driver = webdriver.Firefox(firefox_profile=fp, executable_path=ffdriverpath+'/geckodriver') #firefox driver 다운로드 받고, 다운로드 받은 파일 경로 지정                                                                            

                                            list_of_files = glob.glob(initialdownloadfilepath+'/*') # * means all if need specific format then *.csv
                                            latest_file = max(list_of_files, key=os.path.getctime)

                                            #extract all .zip files on extract_file_path
                                            #print('latest_file = ', latest_file)

                                            zipfilepath = latest_file
                                            with zipfile.ZipFile(zipfilepath, 'r') as zip_ref:
                                                zipfilename = latest_file.split('.')[0]
                                                zipfilename = zipfilename.split('/')[4]
                                                extractedwildfiredir = extractfilepath+'/'+zipfilename
                                                print("extractedwildfiredir = ", extractedwildfiredir)
                                                #os.system('touch '+wildfiredirname)
                                                zip_ref.extractall(extractedwildfiredir)
                                                time.sleep(5)
                                            #read xml files and extract the bounding box coordinates
                                            string_list = []
                                            for xmlfile in os.listdir(extractedwildfiredir):
                                                ext = os.path.splitext(xmlfile)[1]
                                                str1 = ""
                                                if ext == '.xml':
                                                    tree = etree.parse(extractedwildfiredir+'/'+xmlfile)
                                                    westbc = tree.xpath('//westbc/text()')
                                                    eastbc = tree.xpath('//eastbc/text()')
                                                    northbc = tree.xpath('//northbc/text()')
                                                    southbc = tree.xpath('//southbc/text()')
                                                    for ele in northbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in southbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in eastbc:
                                                        str1 += ele
                                                    str1 += ','
                                                    for ele in westbc:
                                                        str1 += ele
                                                    string_list.append(str1)

                                            #print(string_list)
                                            #print(coords_list1)
                                            #file name change
                                            cnt = 0

                                            for i in range(len(coords_list1)):
                                                if i%3 == 0:
                                                    for j in range(len(string_list)):
                                                        if coords_list1[i] == string_list[j]:
                                                            newdirname = coords_list1[3*int(i/3+1)-2]
                                                            #print(newdirname)
                                                            newfilename = newdirname
                                                            newextractedwildfiredir = extractfilepath+'/'+year+'/'+newdirname
                                                            os.system('mv '+extractedwildfiredir+' '+newextractedwildfiredir)                                                        
                                                            time.sleep(20)
                                                        
                                    else:
                                        continue
                                        #print('shapefile size is larger than 100.0km^2')
                                
                                    driver_cnt = 0
                                
                                    #clear list
                                    try:
                                        os.system('rm -r '+extractfilepath+'/lf*')
                                        os.system("rm -r "+extractfilepath+'/'+year+'/'+'lf*')
                                        os.system('rm -r '+extractedwildfiredir+'home')
                                    except:
                                        continue
                                    time.sleep(5)                                    
                                    coords_list1.clear()                                    
                                    print()
                break

for i in range(len(driver.window_handles)):
    driver.switch_to.window(driver.window_handles[0])
    driver.close()
    
print("downloading lcp files completed!!")

2016 ***************************************************************************************************************
driver_cnt =  2
2019 ***************************************************************************************************************


rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/lf*': No such file or directory


downloading lcp files completed!!


# Change LCP files names

In [24]:
expath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'
for year in sorted(os.listdir(expath)):
    if year[0] == '.':
        continue
    for wildfire_name in sorted(os.listdir(expath+'/'+year)):
        if wildfire_name[0] == '.':
            continue
        for lcpfile in sorted(os.listdir(expath+'/'+year+'/'+wildfire_name)):            
            if lcpfile[0] == '.':
                continue
            if lcpfile.split('.')[1] == 'lcp':
                path = expath+'/'+year+'/'+wildfire_name
                #print("path = ", path)                
                try:
                    src_name = lcpfile.split('\\')[0]+'\\\\'+lcpfile.split('\\')[1]
                except:
                    continue
                #print("src_name = ", src_name)
                dst_name = wildfire_name+'.lcp'
                #print("dst_name = ", dst_name)
                os.system('mv '+path+'/'+src_name+" "+path+'/'+dst_name)

print("Changing LCP files' names completed!")

Changing LCP files' names completed!


# Shapefile Projection

In [25]:
import shapefile
from pyproj import Proj, transform
import os, time

#apply projection to all shapefiles
lcp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'

for year in sorted(os.listdir(lcp_path)):
    if year[0] == '.':
        continue
    #print(year,'*********************************************************************')
    wildfire_dir = lcp_path+'/'+year
    for wildfire_name in sorted(os.listdir(wildfire_dir)):
        if wildfire_name[0] == '.':
            continue
        elif(wildfire_name.split('_')[0] == 'ak' or wildfire_name.split('_')[0] == 'AK'):
            continue
        elif(wildfire_name.split('_')[0] == 'hi'):
            continue
        else:
            #print(wildfire_name)
            
            #project shapefiles from EPSG:4326 to EPSG:5070
            shp_dir = wildfire_name    #first folder of the wildfire_name's directory
            #print("shp_dir = ", shp_dir)
            shp_dir_path = wildfire_dir+'/'+wildfire_name
            #print("shp_dir_path = ", shp_dir_path)
            shp_file_path = shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+sorted(os.listdir(shp_dir_path))[0]+'.shp'
            #print("shp_file_path = ", shp_file_path)
            out_shp_file_path = shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+'proj'+sorted(os.listdir(shp_dir_path))[0]+'.shp'
            #print("out_shp_file_path = ", out_shp_file_path)
            
            try:
                #os.system("rm "+shp_dir_path+'/'+"proj*")
                os.system("rm "+shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+'proj'+sorted(os.listdir(shp_dir_path))[0]+'.*')                
            except:
                continue
            
            #pass the dirs containing proj~.* files
            if not os.path.isfile(out_shp_file_path):
                shpf = shapefile.Reader(shp_file_path)    #read shapefile
                wgs_shp = shapefile.Writer(out_shp_file_path, shapefile.POLYGON)    #write shapefile

                fields = shpf.fields
                wgs_fields = wgs_shp.fields

                for name in fields:
                    if type(name) == "tuple":
                        continue
                    else:
                        args = name
                        wgs_shp.field(*args)    #write the source shapefile's fields to the new shapefile

                records = shpf.records()        #record the source shapefile

                for row in records:
                    args = row    
                    wgs_shp.record(*args)       #write the record to the new shapefile

                input_projection = Proj("epsg:4326")
                output_projection = Proj("epsg:5070")

                geom = shpf.shapes()

                #geometry features loop
                for feature in geom:
                    # if there is only one part
                    if len(feature.parts) == 1:
                        # create empty list to store all the coordinates
                        poly_list = []
                        # get each coord that makes up the polygon
                        for coords in feature.points:            
                            x, y = coords[0], coords[1]
                            # tranform the coord
                            new_x, new_y = transform(input_projection, output_projection, y, x)
                            # put the coord into a list structure
                            poly_coord = [float(new_x), float(new_y)]
                            # append the coords to the polygon list
                            poly_list.append(poly_coord)
                        # add the geometry to the shapefile.
                        wgs_shp.poly([poly_list])
                    else:
                        # append the total amount of points to the end of the parts list
                        feature.parts.append(len(feature.points))
                        # enpty list to store all the parts that make up the complete feature
                        poly_list = []
                        # keep track of the part being added
                        parts_counter = 0

                        # while the parts_counter is less than the amount of parts
                        while parts_counter < len(feature.parts) - 1:
                            # keep track of the amount of points added to the feature
                            coord_count = feature.parts[parts_counter]
                            # number of points in each part
                            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
                            # create list to hold individual parts - these get added to poly_list[]
                            part_list = []
                            # cut off point for each part
                            end_point = coord_count + no_of_points

                            # loop through each part
                            while coord_count < end_point:
                                for coords in feature.points[coord_count:end_point]:
                                    x, y = coords[0], coords[1]
                                    # tranform the coord
                                    new_x, new_y = transform(input_projection, output_projection, y, x)
                                    # put the coord into a list structure
                                    poly_coord = [float(new_x), float(new_y)]
                                    # append the coords to the part list
                                    part_list.append(poly_coord)
                                    coord_count = coord_count + 1
                            # append the part to the poly_list
                            poly_list.append(part_list)
                            parts_counter = parts_counter + 1 
                        # add the geometry to to new file 
                        wgs_shp.poly(poly_list)

                wgs_shp.close() #automatically save shapefile

                #write the projection file
                prj = open(out_shp_file_path.split('.')[0]+".prj", "w")            
                epsg_str = 'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]'
                prj.write(epsg_str)
                prj.close()
                #print()

print("Shapefile projection complete!!")

/tmp/ipykernel_14802/1814416355.py:75: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_x, new_y = transform(input_projection, output_projection, y, x)
/tmp/ipykernel_14802/1814416355.py:106: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_x, new_y = transform(input_projection, output_projection, y, x)
/tmp/ipykernel_14802/1814416355.py:75: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_x, new_y = transform(input_projection, output_projection, y, x)
/tmp/ipykernel_14802/1814416355.py:75: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_x, new_y = transform(input_projection, output_projectio

Shapefile projection complete!!


# Generate .input files

In [27]:
import os
from random import *
import datetime as dt

RAWS_num = 80
shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'

def calander(months):
    if months == 2:
        return False
    if months == 4:
        return False
    if months == 6:
        return False
    if months == 9:
        return False
    if months == 11:
        return False
    else:
        return True

def calc_24(hours, RAWS_num):
    hours += RAWS_num * 100
    if hours % 100 == 0:
        hours = hours % 2400 - 41
        if hours < 0:
            return 2400 + hours
        else:
            return hours % 2400
    else:
        return hours % 2400 - 1

for years in sorted(os.listdir(shp_path), key=int):
    for wildfire_name in sorted(os.listdir(shp_path+'/'+years), key=str):
        if(wildfire_name[0] == '.'):
            continue
        elif(wildfire_name.split('_')[0] == 'ak' or wildfire_name.split('_')[0] == 'AK'):
            continue
        elif(wildfire_name.split('_')[0] == 'hi'):
            continue
        else:
            #print()
            year = ''
            months = ''
            days = ''
            hours = ''
            #read from shapefiles
            wildfire_time = sorted(os.listdir(shp_path+'/'+years+'/'+wildfire_name))[0]
            cnt = 0
            for i in wildfire_time:
                if i == '_':
                    cnt += 1
                    continue
                if cnt < 4:
                    year += i
                if cnt >= 4 and cnt < 6:
                    months += i
                if cnt >= 6 and cnt < 8:
                    days += i
                if cnt >= 9:
                    hours += i
                cnt += 1
            #save str wildfire_time to int
            try:
                int_year = int(year)
                int_months = int(months)
                int_days = int(days)
                int_hours = int(hours)
            except ValueError:
                break
                
            #print(wildfire_name)
            #print(wildfire_time)
            #print(int_year, int_months, int_days, int_hours)

            burn_periods = 4
            int_end_hours = calc_24(int_hours, RAWS_num-1)
            end_hours = str(int_end_hours)
            #print("end_hours =", end_hours)
            start_hours = str(int_hours)
            #print("start_time =", start_hours)

            burn_periods_str = ""
            int_burn_days = int_days
            int_burn_months = int_months
            for i in range(burn_periods):
                #month day start_hour end_hour
                if int_burn_days > 30 and calander(int_burn_months) != True:    #did not consider Feburary
                    int_burn_months += 1
                    int_burn_days = 1
                if int_burn_months > 12:
                    int_burn_months = 1

                str1 = str(int_burn_months)+' '+str(int_burn_days)+' '+start_hours+' '+end_hours+'\n'
                burn_periods_str += str1

                int_burn_days += 1

            #print(burn_periods_str)

            RAWS_str = ""
            for i in range(RAWS_num):
                if int_hours >= 2400:
                    int_days += 1
                    int_hours %= 2400
                if int_days > 30 and calander(int_months) != True:    #did not consider Feburary
                    int_months += 1
                    int_days = 1
                if int_months > 12:
                    int_year += 1
                    int_months = 1
                    
                #year month day hour temperature humidity percipitation wind_speed wind_direction cloud_cover
                str2 = str(int_year)+' '+str(int_months)+' '+str(int_days)+' '+str(int_hours)+' '+str(randrange(50, 86))+' '+str(randrange(10, 81))+' '+str(0.00)+' '+str(randrange(2, 15))+' '+str(randrange(0, 360))+' '+str(randrange(0, 66))+'\n'
                RAWS_str += str2
                int_hours += 100
                
            #print(RAWS_str)
            if (int_end_hours+1 ) % 60 == 0:
                int_end_hours = int_end_hours + 1 + 40

            input_string = "FARSITE INPUTS FILE VERSION 1.0\n\
FARSITE_START_TIME: "+months+' '+days+' '+start_hours+"\n\
FARSITE_END_TIME: "+str(int_burn_months)+' '+str(int_burn_days-1)+' '+str(int_end_hours)+"\n\
FARSITE_TIMESTEP: 60\n\
FARSITE_DISTANCE_RES: 30.0\n\
FARSITE_PERIMETER_RES: 60.0\n\
FARSITE_MIN_IGNITION_VERTEX_DISTANCE: 15.0\n\
FARSITE_SPOT_GRID_RESOLUTION: 15.0\n\
FARSITE_SPOT_PROBABILITY: 0.05\n\
FARSITE_SPOT_IGNITION_DELAY: 0\n\
FARSITE_MINIMUM_SPOT_DISTANCE: 30\n\
FARSITE_ACCELERATION_ON: 1\n\
FARSITE_FILL_BARRIERS: 1\n\
SPOTTING_SEED: 253114\n\
\n\
FARSITE_BURN_PERIODS: "+str(burn_periods)+"\n\
"+burn_periods_str+"\n\
\n\
FUEL_MOISTURES_DATA: 1\n\
0 3 4 6 70 100\n\
RAWS_ELEVATION: 4159\n\
RAWS_UNITS: English\n\
\n\
RAWS: "+str(RAWS_num)+"\n\
"+RAWS_str+"\n\
\n\
FOLIAR_MOISTURE_CONTENT: 100\n\
CROWN_FIRE_METHOD: Finney\n\
NUMBER_PROCESSORS: 1\n\
\n\
\n\
FLAMELENGTH:\n\
SPREADRATE:\n\
INTENSITY:\n\
CROWNSTATE:\n\
"

            #print(input_string)
            inputfilepath = shp_path+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+wildfire_time+".input"

            f = open(inputfilepath, "w")
            f.write(input_string)
            f.close()

print("Generating .input file completed!")

Generating .input file completed!


# Generate .txt files

In [28]:
import os

shppath = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
lcppath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

for years in sorted(os.listdir(shppath), key=int):
    if years[0] == '.':
        continue
    for wildfire_name in sorted(os.listdir(shppath+'/'+years), key=str):
        if wildfire_name[0] == '.':
            continue
        elif(wildfire_name.split('_')[0] == 'ak' or wildfire_name.split('_')[0] == 'AK'):
            continue
        elif(wildfire_name.split('_')[0] == 'hi'):
            continue
        else:
            #print(wildfire_name)
            wildfire_time = sorted(os.listdir(shppath+'/'+years+'/'+wildfire_name))[0]
            outputdatadir = shppath+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+'output'
            #print(outputdatadir)

            #create output data directory
            if not os.path.isdir(outputdatadir): 
                os.system("mkdir "+outputdatadir)
                print("outputdata directory creation completed!")
            
            shpfilepath = shppath+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+"proj"+wildfire_time+".shp"
            lcpfilepath = lcppath+'/'+years+'/'+wildfire_name+"/clipped.lcp"
            inputfilepath = shppath+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+wildfire_time+".input"
            barrierfilepath = '0'

            #.lcp = clipped.lcp / .input = wildfire_name.input / .shp = proj+wildfire_time.shp / barrier = 0 / output dir / 0
            txt_string = lcpfilepath+' '+inputfilepath+' '+shpfilepath+' '+barrierfilepath+' '+outputdatadir+' '+str(0)

            txtfilepath = shppath+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+"inputtxt.txt"

            f = open(txtfilepath, "w")
            f.write(txt_string)
            f.close()

print("execute text file creation completed!")

execute text file creation completed!


# Clip LCP files

In [96]:
from osgeo import gdal
import rasterio
import os
import time

lcpfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

for year in sorted(os.listdir(lcpfilepath), key=int):
    #hprint(year, '***************************************************************************************************************')    
    for target_fire in sorted(os.listdir(lcpfilepath+'/'+year), key=str):
        path = lcpfilepath+'/'+year        
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi'):
            continue
        else:
            #print(target_fire)
            #check if there is the lcp file
            if os.path.isfile(lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".lcp"):
                os.chdir(lcpfilepath+'/'+year+'/'+target_fire)
                #remove all previous clipped files
                try:
                    os.system("rm clipped.*")
                    #os.system("rm *.tif")
                except:
                    continue
                
                lcp_path = lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".lcp"
                tif_path = lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".tif"
                #open the lcp file
                lcp = rasterio.open(lcp_path)
                lcp_centerx = (lcp.bounds[0]+lcp.bounds[2])/2    #centerx
                lcp_centery = (lcp.bounds[1]+lcp.bounds[3])/2    #centery
                new_x1 = lcp_centerx - 9000
                new_x2 = lcp_centerx + 9000
                new_y1 = lcp_centery - 9000
                new_y2 = lcp_centery + 9000
                out_tifpath = lcpfilepath+'/'+year+'/'+target_fire+'/clipped.tif'
                out_lcppath = lcpfilepath+'/'+year+'/'+target_fire+'/clipped.lcp'
                
                #print("extent =", new_x1, new_y1, new_x2, new_y2)
                
                #clip the lcp files
                #os.system("gdal_translate "+lcp_path+" "+tif_path)
                #time.sleep(0.05)
                #os.system("gdal_translate -projwin "+str(new_x1)+" "+str(new_y2)+" "+str(new_x2)+" "+str(new_y1)+" -f GTiff "+tif_path+" "+out_tifpath)
                os.system("gdal_translate -projwin "+str(new_x1)+" "+str(new_y2)+" "+str(new_x2)+" "+str(new_y1)+" -f GTiff "+lcp_path+" "+out_lcppath)
                #time.sleep(0.05)
                #os.system("gdal_translate "+tif_path+" "+out_lcppath)
                #print("src lcp path = ", lcp_path)
                #print("dst lcp path = ", out_lcppath)
                #print()

print("clipping lcp files completed!!")

Input file size is 767, 776
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 766, 774
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 767, 776
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 769, 779
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 766, 774
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 710, 722
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 714, 725
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 714, 726
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 804, 814
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 811, 821
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 788, 793
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 79

# Modify headers of LCP files

In [21]:
#header value got corrupted
import rasterio
import numpy as np

lcpfilepath1 = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/clipped.lcp'
lcpfilepath2 = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/al_caney_head.lcp'
lcpfilepath3 = '/home/qweddww123/farsite/examples/Panther/test1177973/input/test1177973.lcp'
lcpfilepath4 = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_half_way/al_half_way.lcp'

lcp1 = rasterio.open(lcpfilepath1)
lcp2 = rasterio.open(lcpfilepath2)
lcp3 = rasterio.open(lcpfilepath3)
lcp4 = rasterio.open(lcpfilepath4)

#read lcp file header data
with open(lcpfilepath1, 'rb') as fid:
    data_array1 = np.fromfile(fid, np.int16)    #al_caney_head_clipped
with open(lcpfilepath2, 'rb') as fid:
    data_array2 = np.fromfile(fid, np.int16)    #al_caney_head
with open(lcpfilepath3, 'rb') as fid:
    data_array3 = np.fromfile(fid, np.int16)    #test1177973
with open(lcpfilepath4, 'rb') as fid:
    data_array4 = np.fromfile(fid, np.int16)    #al_half_way


#print(lcp1.width)
#print(lcp2.width)
    
print(data_array1)
print(data_array2)

#print(lcp1.read(8))
#print(lcp2.read(8))

#print(len(data_array1))
#print(len(data_array2))

#save header for the target clipped 600*600 lcp


#fwrite the header to the clipped lcp file


#print(len(data_array1) - 600*600*8)

#for i in range(2105, 2505):
#    print(data_array1[i])
#for i in range(3658, 4058):
#    print(data_array2[i])

'''
for i in range(len(data_array2)):
    if i < len(data_array2):
        for j in range(len(lcp2.read(1)[0])):
            if j < len(lcp2.read(1)[0]):
                if data_array2[i] == lcp2.read(1)[0][j] and data_array2[i+1] == lcp2.read(1)[0][j+1]:
                    print(i)


print(len(data_array2) - lcp2.width*lcp2.height*8)
print(lcp2.read(1)[0])
cnt = 0
for i in range(len(data_array2)):    
    if data_array2[i] == 181:
        print(i)


#print(lcp3.read(1)[0])
for i in range(len(data_array3)):
    if i < len(data_array3):
        if data_array3[i] == 849:
            print(i)
'''
print(len(data_array1)-lcp1.width*lcp1.height*8)
print(lcp1.width, lcp1.height)
print(len(data_array2)-lcp2.width*lcp2.height*8)
print(len(data_array3)-lcp3.width*lcp3.height*8)
print(len(data_array4)-lcp4.width*lcp4.height*8)

[ 21   0  20 ... 190 100   1]
[ 21   0  20 ... 190 100   1]
3658
600 600
3658
3658
3658


0.5754510661563697

In [97]:
#run FARSITE from /home/qweddww123/farsite/src
#./TestFARSITE {input txt file path}

# Generate LCP files for clipping (skip 가능）

In [20]:
lcpfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

for year in sorted(os.listdir(lcpfilepath), key=int):
    print(year, '***************************************************************************************************************')    
    for target_fire in sorted(os.listdir(lcpfilepath+'/'+year), key=str):
        path = lcpfilepath+'/'+year        
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi'):
            continue
        else:
            #print(target_fire)
            #check if there is the lcp file
            if os.path.isfile(lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".lcp"):
                os.chdir(lcpfilepath+'/'+year+'/'+target_fire)
                #remove all previous clipped files
                try:
                    os.system("rm clipped.*")
                    #os.system("rm *.tif")
                except:
                    continue
                
                lcp_path = lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".lcp"
                tif_path = lcpfilepath+'/'+year+'/'+target_fire+'/'+target_fire+".tif"
                #open the lcp file
                lcp = rasterio.open(lcp_path)
                lcp_centerx = (lcp.bounds[0]+lcp.bounds[2])/2    #centerx
                lcp_centery = (lcp.bounds[1]+lcp.bounds[3])/2    #centery
                new_x1 = lcp_centerx - 9000
                new_x2 = lcp_centerx + 9000
                new_y1 = lcp_centery - 9000
                new_y2 = lcp_centery + 9000
                out_tifpath = lcpfilepath+'/'+year+'/'+target_fire+'/clipped.tif'
                out_lcppath = lcpfilepath+'/'+year+'/'+target_fire+'/clipped.lcp'
                
                #print("extent =", new_x1, new_y1, new_x2, new_y2)
                
                #clip the lcp files
                #os.system("gdal_translate "+lcp_path+" "+tif_path)
                #time.sleep(0.05)
                #os.system("gdal_translate -projwin "+str(new_x1)+" "+str(new_y2)+" "+str(new_x2)+" "+str(new_y1)+" -f GTiff "+tif_path+" "+out_tifpath)
                os.system("gdal_translate -projwin "+str(new_x1)+" "+str(new_y2)+" "+str(new_x2)+" "+str(new_y1)+" -f GTiff "+lcp_path+" "+out_lcppath)
                #time.sleep(0.05)
                #os.system("gdal_translate "+tif_path+" "+out_lcppath)
                #print("src lcp path = ", lcp_path)
                #print("dst lcp path = ", out_lcppath)
                #print()

print("clipping lcp files completed!!")

2016 ***************************************************************************************************************
Input file size is 767, 776
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 766, 774
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 767, 776
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 769, 779
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 766, 774
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 710, 722
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 714, 725
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 714, 726
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 804, 814
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 811, 821
0...10...20...30...40...50...60...70...80...90...100 -

# Create SHP files for clipping from clipped LCPs　（skip 가능）

Clip LCP -> create SHP from clipped LCP -> clip original LCP by SHP

In [50]:
import shapefile
import geopandas as pd
import rasterio
import os

shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
lcpfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

for years in sorted(os.listdir(lcpfilepath), key=int):
    for wildfire_name in sorted(os.listdir(lcpfilepath+'/'+years), key=str):
        if(wildfire_name[0] == '.'):
            continue
        elif(wildfire_name.split('_')[0] == 'ak' or wildfire_name.split('_')[0] == 'AK'):
            continue
        elif(wildfire_name.split('_')[0] == 'hi'):
            continue
        else:
            #print(lcpfilepath+'/'+years+'/'+wildfire_name)
            #print(shp_path+'/'+years+'/'+wildfire_name)
            #print(sorted(os.listdir(shp_path+'/'+years+'/'+wildfire_name), key=str)[0])
            wildfire_time = sorted(os.listdir(shp_path+'/'+years+'/'+wildfire_name), key=str)[0]
            try:
                os.system("rm "+shp_path+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+"shptest.*")
            except:
                continue

            #read bounding box coordinates of lcp files
            lcp_path = lcpfilepath+'/'+years+'/'+wildfire_name+'/'+"clipped.lcp"
            #open the lcp file
            lcp = rasterio.open(lcp_path)
            x1 = lcp.bounds[0]
            y1 = lcp.bounds[1]
            x2 = lcp.bounds[2]
            y2 = lcp.bounds[3]
            #print(x1, y1, x2, y2)

            w = shapefile.Writer(shp_path+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+"shptest.shp")
            #print(shp_path+'/'+years+'/'+wildfire_name+'/'+wildfire_time+'/'+"shptest.shp")
            w.field('name', 'C')
            w.poly([
                [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
                #[[113,24], [112,32], [117,36], [122,37], [118,20]]
            ])
            w.record('polygon')
            w.close()

# Clip LCP file by SHP file

In [94]:
import os
import rasterio
from shapely.geometry import box
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

#shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
#lcpfilepath = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

shp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head/20160321_0000/shptest.shp'
fp = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/al_caney_head.lcp'
out_lcp = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/'+"cliptest1.lcp"

os.system('rm /home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract/2016/al_caney_head/'+"cliptest1.lcp")

data = rasterio.open(fp)
#show((data, 4), cmap='terrain')

shpfile = gpd.read_file(shp_path)
x1 = float(shpfile.bounds.minx[0])
y1 = float(shpfile.bounds.miny[0])
x2 = float(shpfile.bounds.maxx[0])
y2 = float(shpfile.bounds.maxy[0])

minx, miny = x1, y1
maxx, maxy = x2, y2

bbox = box(minx, miny, maxx, maxy)

geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs="EPSG:4326")

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

coords = getFeatures(geo)
#print(coords)

out_img, out_transform = mask(data, shapes=coords, crop=True)

out_meta = data.meta.copy()
#print(out_meta)

epsg_code = 4326

out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
               )

with rasterio.open(out_lcp, "w", **out_meta) as dest:
    dest.write(out_img)

clipped = rasterio.open(out_lcp)
#show((clipped, 4), cmap='terrain')

/home/qweddww123/anaconda3/envs/pygdal/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/qweddww123/anaconda3/envs/pygdal/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [93]:


#gdal.Warp -s_srs EPSG:4326 -t_srs EPSG:4326 -of GTiff -cutline 